In [1]:
import pandas as pd
import os
import unittest
import string
import re
import copy
pd.set_option('display.max_rows', 10)

In [2]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day22_input.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s = file1.read().splitlines()
pd.Series(s)

0     Player 1:
1            24
2            22
3            26
4             6
        ...    
48           38
49           35
50            2
51           33
52           31
Length: 53, dtype: object

In [3]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day22_example1.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s_ex1 = file1.read().splitlines()
pd.Series(s_ex1)

0     Player 1:
1             9
2             2
3             6
4             3
        ...    
8             5
9             8
10            4
11            7
12           10
Length: 13, dtype: object

In [4]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day22_example2.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file2 = open(trainFile, 'r') 
s_ex2 = file2.read().splitlines()
pd.Series(s_ex2)

0    Player 1:
1           43
2           19
3             
4    Player 2:
5            2
6           29
7           14
dtype: object

In [5]:
def parse(input):
    data = []
    deck = []
    for line in input:
        if line.count("Player") > 0:
            if line.count("Player 2") > 0:
                data.append(deck)
                deck = []
        elif len(line.strip()) > 0:
            deck.append(int(line))
    data.append(deck)
    return data

In [6]:
p_ex1 = parse(s_ex1)
p_ex1

[[9, 2, 6, 3, 1], [5, 8, 4, 7, 10]]

In [7]:
p = parse(s)
p

[[24,
  22,
  26,
  6,
  14,
  19,
  27,
  17,
  39,
  34,
  40,
  41,
  23,
  30,
  36,
  11,
  28,
  3,
  10,
  21,
  9,
  50,
  32,
  25,
  8],
 [48,
  49,
  47,
  15,
  42,
  44,
  5,
  4,
  13,
  7,
  20,
  43,
  12,
  37,
  29,
  18,
  45,
  16,
  1,
  46,
  38,
  35,
  2,
  33,
  31]]

In [8]:
def playGame(input):
    first = input[0][:]
    second = input[1][:]
    #print(first, second)
    while len(first) > 0 and len(second) > 0:
        topCardfirst = first.pop(0)
        topCardsecond = second.pop(0)
        if topCardfirst > topCardsecond:            
            first.append(topCardfirst)
            first.append(topCardsecond)
        else:
            second.append(topCardsecond)
            second.append(topCardfirst)
        #print(first, second)
    return (first,second)

In [9]:
playGame(p_ex1)

([], [3, 2, 10, 6, 8, 5, 9, 4, 7, 1])

In [10]:
def score(input):
    result = 0
    multiplier = len(input)
    for card in input:
        result += card * multiplier
        multiplier -= 1
    return result

In [11]:
sum(map(score, playGame(p_ex1)))

306

In [12]:
sum(map(score, playGame(p)))

35562

In [13]:
def toHashKey(input):    
    return "|".join([",".join(map(str,input[0])), ",".join(map(str, input[1]))])

def playGameRecursive(data):
    input=data[:]
    cache = {}
    debug = False #len(cache)> 550:
    if debug:
        print("Starting new game with", input)
    first = input[0][:]
    second = input[1][:]
    while len(first) > 0 and len(second) > 0:
        currentGame = (first, second)
        key = toHashKey(currentGame)
        if key in cache:            
            firstWins = True # break recursion by letting player one win
            print("found result in cache", len(first)+len(second), key)
            return (first, second, True)
        
        cache[key] = True
        topCardfirst = first.pop(0)
        topCardsecond = second.pop(0)
        firstWins = True

        if len(first) >= topCardfirst and len(second) >= topCardsecond:
            if debug:
                "starting subgame"
            subGameResult = playGameRecursive((first[0:topCardfirst], second[0:topCardsecond]))
            firstWins = len(subGameResult[0]) > 0 or subGameResult[2]
        else:
            firstWins = topCardfirst > topCardsecond 
        
        if firstWins:
            first.append(topCardfirst)
            first.append(topCardsecond)
        else:
            second.append(topCardsecond)
            second.append(topCardfirst)
        if debug:
            print("New state",first, second)
    if debug:
        print("Finished game with" , result)
    return (first, second, False)

In [14]:
playGameRecursive(p_ex1)

([], [7, 5, 6, 2, 4, 1, 10, 8, 9, 3], False)

In [15]:
a_ex1 = playGameRecursive(p_ex1)
score(a_ex1[0]) + score(a_ex1[1])

291

In [16]:
p_ex2 = parse(s_ex2)
p_ex2

[[43, 19], [2, 29, 14]]

In [17]:
playGameRecursive(p_ex2)

found result in cache 5 43,19|2,29,14


([43, 19], [2, 29, 14], True)

In [19]:
a = playGameRecursive(p)

 in cache 29 49,6,48,4,36,27,35,23,40,16,45,11,43,18,47,14,33,12,50,8,44,2,29,25,28,20|3,39,17
found result in cache 39 43,20,49,22,46,13,44,19,29,17,42,12,41,2,40,18,45,21,32,14,47,27,33,3,31,7,50,4,48,30,26,25|23,34,1,38,5,35,24
found result in cache 25 11,49,4,36,1,45,25,38,5,43,27,47,21,50,12,40,7|46,8,33,3,48,13,32,24
found result in cache 11 43,25,45,2,38,32|12,46,11,33,4
found result in cache 15 1,33,25,47,5,39,32,46,12,43,2|44,6,34,26
found result in cache 13 26,47,6,46,22|39,25,44,2,48,34,40,33
found result in cache 19 42,7,35,23,31,28|20,40,19,44,16,29,25,36,17,48,5,41,22
found result in cache 17 20,45,3,41,8,38,19,33,4,42,29,34,11|44,27,32,24
found result in cache 19 3,42,19,41,31,27,9,43,4,45,1|34,20,38,29,44,16,33,32
found result in cache 11 50,11,44,9,41,22,28,12|1,39,3
found result in cache 7 44,21,50,22|7,41,28
found result in cache 9 4,50,22,44,7,48,33|41,3
found result in cache 29 23,40,7,39,24,47,9,44,1,46,31,38,20,50,17,42,11,36,5,49,19,34,2,45,27|43,16,32,29
found 

In [20]:
score(a[0]) 
# 5910 is too low
# 9428 is too low

34424